# GU4260 Problem Set 7
This notebook contains some useful starting code for Problem Set 7

First, we'll define the limb-darkening function and the two functions (numerator and denominator) that we need to integrate, as in the previous problem set.

In [1]:
import math
import numpy as np
import my_transit

def I(r):
    '''A Limb-darkening function'''
    mu = (1 - (r**2))**(0.5)
    return 1 - (1 - (mu)**0.5)

def func1(r, p, z):
    '''Numerator for the transit integral'''
    return I(r) * (1 - my_transit.delta(p,r,abs(z))) * 2 * r

def func2(r, p, z):
    '''Denominator for the transit integral'''
    return I(r) * 2 * r

Next, define the function to do the integration necessary to compute the predicted transit light curve given the three parameters (p, t0, tau). We'll use scipy's Gaussian quadrature to the integration as this very efficient.

In [2]:
from scipy.integrate import quad
def ComputeLightCurve(time, p, t0, tau):
    """ Convert from Time to z and compute predicted transit curve. Given the three
        parameters p, t0, and tau as well as an array containing the time values,
        returns the predicted (normalized) transit flux values as an array"""
    zs = (time-t0)/tau
    FluxRatio = []
    for zval in zs:
        FluxObscured, FO_error = quad(func1, 0.0, 1.0, epsrel=1e-06, args=(p,zval))
        FluxUnobscured, FU_error = quad(func2, 0.0, 1.0, epsrel=1e-06, args=(p,zval))
        FluxRatio.append(FluxObscured/FluxUnobscured)
    return np.array(FluxRatio)

Read in the Kepler observed transit curve (or replace with your own code using kplr)

In [3]:
time, flux, flux_err = np.loadtxt("KOI97.01_1.out", unpack=True)

Compute the predicted transit curve given some parameters and the whole time curve.  Note that his section definitely needs to be replaced by code which extracts the required section and normalizes the observed flux values (as described in the problem set) -- I include this just to show an example of how to call ComputeLightCurve.

In [4]:
p = 0.0775
tau = 0.1
t0 = 124.51

FluxRatioTheory = ComputeLightCurve(time, p, t0, tau)